In [ ]:
import pandas as pd
import torch

import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, num_node_features=23, hidden_dim=10):
        super(Net, self).__init__()
        self.conv1 = GCNConv(num_node_features, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.ln = torch.nn.Linear(hidden_dim, 1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        # x = F.dropout(x)
        x = self.conv2(x, edge_index)

        x = x.mean(axis=-2)
        x = self.ln(x)
        x = F.sigmoid(x)
        return x

In [ ]:
filename = "data/1350/D/188663388.cpp"
base_filename = filename.split(".")[0]

In [ ]:
# node features
df_features = pd.read_csv(base_filename + "-features.txt")
df_features["IsMain"] = (df_features["FunctionName"] == "main").astype(int)
node_features = df_features.drop(
    ["FunctionName", "BasicBlockIndex", "BasicBlockName", "depth", "loopNestLevel"],
    axis=1,
)
node_features = torch.Tensor(node_features.values)

# edge features
with open(base_filename + "-cfg.txt", mode="r") as f:
    lines = f.readlines()
s_nodes = torch.Tensor([int(ln.split(" ")[0]) for ln in lines]).to(torch.int)
t_nodes = torch.Tensor([int(ln.split(" ")[1][:-1]) for ln in lines]).to(torch.int)
edges = torch.stack([s_nodes, t_nodes])

In [ ]:
df_stats = pd.read_csv(base_filename + ".csv")

df_stats["insert_prob"] = df_stats.afl_banner.apply(
    lambda x: x.split("/")[-1].split("_")[1] if "vanila" not in x else 0
)
df_stats["insert_random_seed"] = df_stats.afl_banner.apply(
    lambda x: x.split("/")[-1].split("_")[2].split(".")[0] if "vanila" not in x else 0
)
df_stats = df_stats[
    [
        "afl_banner",
        "insert_prob",
        "insert_random_seed",
        "edges_found",
        "total_edges",
        "bitmap_cvg",
        "saved_crashes",
    ]
]
df_stats["bitmap_cvg"] = df_stats["bitmap_cvg"].apply(lambda x: float(x[:-1]))
df_stats = df_stats.sort_values(["insert_prob", "insert_random_seed"])

df_groundtruth = df_stats.iloc[1:, :]
df_groundtruth.insert(
    df_groundtruth.shape[1],
    "relative_bitmap_cvg",
    (df_groundtruth["bitmap_cvg"] / df_stats.iloc[0, :]["bitmap_cvg"]).apply(
        lambda x: min(1.0, x)
    ),
)

In [ ]:
block_files = [
    "data/1350/D/188663388/sleep_0.01_3.block",
    "data/1350/D/188663388/sleep_0.01_4.block",
]

data_list = []
for bf in block_files:
    with open("data/1350/D/188663388/sleep_0.01_3.block", mode="r") as f:
        lines = f.readlines()
    inserted_blocks = torch.Tensor([int(ln[:-1]) for ln in lines]).to(torch.int)
    is_inserted = torch.zeros(node_features.shape[0], 1)
    is_inserted[inserted_blocks] = 1
    data = Data(
        x=torch.concatenate([node_features, is_inserted], axis=1),
        edge_index=edges,
        y=torch.Tensor([0.01]),
    )
    data_list.append(data)

In [ ]:
data

In [ ]:
model = Net()
model(dataset_lists[0])